# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import pandas as pd


data = {
    "company": [
        "Social Bicycles Inc."
    ],
    "gbfs_href": "https://hamilton.socialbicycles.com/opendata/gbfs.json",
    "href": "/v2/networks/sobi-hamilton",
    "id": "sobi-hamilton",
    "location": {
        "city": "Hamilton, ON",
        "country": "CA",
        "latitude": 43.25643601915583,
        "longitude": -79.86929655075073
    },
    "name": "SoBi"
}

# Get the gbfs_href from the JSON data
gbfs_href = data['gbfs_href']

# Send a GET request to the gbfs_href URL
response = requests.get(gbfs_href)
gbfs_data = response.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [2]:
# Retrieve the station information and status URLs
feeds = gbfs_data['data']['en']['feeds']
station_info_href = None
station_status_href = None
for feed in feeds:
    if feed['name'] == 'station_information':
        station_info_href = feed['url']
    elif feed['name'] == 'station_status':
        station_status_href = feed['url']
    if station_info_href and station_status_href:
        break

if not station_info_href or not station_status_href:
    print('Bike locations data not found')
    exit()

# Send GET requests to the station information and status URLs
response_info = requests.get(station_info_href)
response_status = requests.get(station_status_href)
station_info_data = response_info.json()
station_status_data = response_status.json()

Put your parsed results into a DataFrame.

In [3]:
# Send GET requests to the station information and status URLs
response_info = requests.get(station_info_href)
response_status = requests.get(station_status_href)
station_info_data = response_info.json()
station_status_data = response_status.json()

# Extract relevant data from station information and status
station_info = station_info_data['data']['stations']
station_status = station_status_data['data']['stations']

# Create a DataFrame to store the combined information
data = []
for info in station_info:
    station_id = info['station_id']
    name = info['name']
    latitude = info['lat']
    longitude = info['lon']
    status = next((status for status in station_status if status['station_id'] == station_id), {})
    num_bikes_available = status.get('num_bikes_available', 'N/A')
    data.append([station_id, name, latitude, longitude, num_bikes_available])

df = pd.DataFrame(data, columns=['Station ID', 'Name', 'Latitude', 'Longitude', 'Number of Bikes Available'])
print(df)

df.to_csv('bike_station_data.csv', index=False)

# print(f"Station ID: {station_id}")
# print(f"Name: {name}")
# print(f"Latitude: {latitude}")
# print(f"Longitude: {longitude}")
# print(f"Number of bikes available: {num_bikes_available}")
# print()



    Station ID                     Name   Latitude  Longitude  \
0      hub_435                Gore Park  43.256317 -79.868664   
1      hub_436  James North at Mulberry  43.261819 -79.866743   
2      hub_527                Seedworks  43.259111 -79.863613   
3      hub_530  Cootes at York and King  43.265581 -79.949656   
4      hub_535           Osler at Grant  43.259550 -79.942643   
..         ...                      ...        ...        ...   
139   hub_5845           Jackson Square  43.256935 -79.869577   
140   hub_5846         Cannon at Steven  43.256461 -79.849981   
141   hub_5883          RBG Rock Garden  43.289479 -79.891693   
142   hub_5903                   Pier 8  43.275289 -79.856907   
143   hub_6148    King William at James  43.257392 -79.867902   

     Number of Bikes Available  
0                            7  
1                            2  
2                            2  
3                           11  
4                            1  
..                   